# Scoring Analysis Notebook

This notebook imports all `calcular_` functions from `get_conver_scores.py` and displays dataframes for each metric.


In [1]:
import sys
import os
import pandas as pd
from IPython.display import display as iDisplay

# Add the parent directory to the path to access app.services
parent_dir = os.path.dirname(os.getcwd())
sys.path.insert(0, parent_dir)

# Add the scoring_scripts directory to the path
# From scoring_notebooks/, go up one level and into scoring_scripts/
sys.path.insert(0, os.path.join(parent_dir, 'scoring_scripts'))

# Import from the correct module path
from app.services.scoring_service import read_msg

# Alternative: if the above doesn't work, uncomment the line below
# sys.path.insert(0, '../scoring_scripts')

from get_conver_scores import (
    calcular_muletillas,
    calcular_claridad,
    calcular_participacion_dinamica,
    calcular_cobertura_temas_json,
    calcular_indice_preguntas,
    calcular_ppm_variabilidad, 
    calcular_objetivo_principal,
    get_conver_scores
)


## Input Transcript

Using the demo transcript from `get_conver_scores.py`


In [2]:
transcript_demo = [
    {
        "speaker": "vendedor", 
        "text": "Hola, buenas tardes. Bienvenido a nuestra exposición virtual, eh... mi nombre es Carlos. Veo que se ha interesado justo por el nuevo Conversa XL a través de la web. Tiene buen ojo, es la unidad que acabamos de recibir esta misma mañana y ya está disponible para reserva inmediata.", 
        "duracion": 18
    },
    {
        "speaker": "cliente", 
        "text": "Hola Carlos. Sí, la verdad es que estaba buscando algo más grande porque la familia ha crecido y mi coche actual se nos ha quedado minúsculo.", 
        "duracion": 12
    },
    {
        "speaker": "vendedor", 
        "text": "Le entiendo perfectamente. El espacio es vital. Déjeme decirle que hemos cambiado totalmente el **mindset** de diseño para enfocarnos en familias como la suya. Fíjese en las fotos del catálogo que le acabo de compartir; esas líneas no solo son estética, son refuerzos de acero al boro. En seguridad somos líderes: 5 estrellas Euro NCAP para que no tenga ningún miedo al llevar a sus hijos.", 
        "duracion": 38
    },
    {
        "speaker": "cliente", 
        "text": "La seguridad es importante, claro. Pero mi esposa está obsesionada con el maletero. En el que tenemos ahora, meter el carrito del bebé y la compra es imposible, siempre tenemos que dejar bolsas en los asientos de atrás.", 
        "duracion": 18
    },
    {
        "speaker": "vendedor", 
        "text": "Comprendo. Si le sigo bien, lo que me está diciendo es que su mayor dolor de cabeza actual es la falta de capacidad de carga y necesita garantías de que podrá meter el carrito y las bolsas del supermercado todo junto en el maletero sin invadir los asientos, ¿es eso correcto?", 
        "duracion": 25
    },
    {
        "speaker": "cliente", 
        "text": "Exacto, eso es justo lo que necesito. Que no sea un tetris cada vez que salimos de viaje.", 
        "duracion": 8
    },
    {
        "speaker": "vendedor", 
        "text": "Pues mire los datos técnicos que le envío. Tenemos 650 litros de capacidad real. Aquí le caben dos carritos si hace falta. Además, la boca de carga es muy baja para que no se deje la espalda levantando peso. Y los asientos traseros son individuales; no va a tener problema para colocar tres sillas infantiles.", 
        "duracion": 35
    },
    {
        "speaker": "cliente", 
        "text": "Oye, pues es verdad que se ve inmenso. ¿Y de tecnología qué tal va? Porque no quiero algo que sea muy complicado de usar, la pantalla esa parece una nave espacial.", 
        "duracion": 15
    },
    {
        "speaker": "vendedor", 
        "text": "Parece compleja, pero el **feedback** que recibimos es que se aprende a usar en cinco minutos. Mire, le voy a ser sincero, la conectividad hoy en día nos facilita la vida y usted necesita gestionar las llamadas y el mapa por voz, sin soltar el volante en ningún momento bajo ninguna circunstancia.", 
        "duracion": 40
    },
    {
        "speaker": "cliente", 
        "text": "Ya, mientras no se cuelgue... ¿Y el motor? Hago muchos kilómetros para ir al trabajo y la gasolina está carísima.", 
        "duracion": 10
    },
    {
        "speaker": "vendedor", 
        "text": "No se preocupe por eso. Montamos un motor híbrido auto-recargable. El coche gestiona solo cuándo usar la batería. En ciudad va a ir casi siempre en eléctrico, reduciendo el gasto de combustible a la mitad comparado con su coche actual. Es eficiencia pura.", 
        "duracion": 28
    },
    {
        "speaker": "cliente", 
        "text": "Suena bien lo del ahorro. Pero vamos a lo doloroso... he estado mirando el modelo similar de la marca alemana y se me va de precio. Imagino que este, siendo nuevo y con tanta tecnología, costará un ojo de la cara.", 
        "duracion": 18
    },
    {
        "speaker": "vendedor", 
        "text": "Para nada, ahí es donde el Conversa XL brilla. Sabemos que el **budget** familiar es sagrado. Al ser una gestión online, nuestro precio final está actualmente un 12% por debajo de la competencia directa. Básicamente, se lleva más coche por menos dinero.", 
        "duracion": 32
    },
    {
        "speaker": "cliente", 
        "text": "Un 12% es bastante diferencia... ¿Y tenéis financiación? Porque no quería descapitalizarme ahora mismo pagándolo todo de golpe.", 
        "duracion": 12
    },
    {
        "speaker": "vendedor", 
        "text": "Sí, tenemos un plan flexible totalmente digital. Podemos ajustar la entrada y dejar una cuota muy cómoda. De hecho, si lo tramitamos ahora por el portal, le incluyo el envío a domicilio sin coste adicional.", 
        "duracion": 22
    },
    {
        "speaker": "cliente", 
        "text": "Pues con ese descuento y el envío a casa me habéis convencido. Me cuadra todo. ¿Qué tengo que hacer para confirmar la compra ahora mismo?", 
        "duracion": 12
    },
    {
        "speaker": "vendedor", 
        "text": "¡Fantástico! Le acabo de enviar un enlace seguro a su correo. Solo tiene que subir una foto de su DNI y completar el formulario de la financiera. En cuanto lo reciba, bloqueamos el coche para usted y empezamos con la gestión del envío.", 
        "duracion": 25
    }
]



In [3]:
transcript_db = read_msg('8ab12953-67b4-4d1e-a740-5f7859011ea5')
import json
#print(json.dumps(transcript_demo, indent=2, ensure_ascii=False))

/Users/dfloren10/Code/Back-End-LLM/app/services/scoring_service.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


## Calculate All Metrics


In [7]:
# Calculate all metrics

transcript = transcript_db 

muletillas_result = calcular_muletillas(transcript)
claridad_result = calcular_claridad(transcript)
participacion_result = calcular_participacion_dinamica(transcript)
cobertura_temas_result = calcular_cobertura_temas_json(transcript)
preguntas_result = calcular_indice_preguntas(transcript)
ppm_result = calcular_ppm_variabilidad(transcript)
obj_result = calcular_objetivo_principal(transcript)


## DataFrames for Each Metric


In [8]:
df_muletillas = pd.DataFrame([{
    "puntuacion": muletillas_result["puntuacion"], 
    "penalizacion": muletillas_result["penalizacion"],
    "total_muletillas": muletillas_result["total_muletillas"],
    "repeticion_constante": muletillas_result.get("repeticion_constante", None),
    "porcentaje_repeticion": muletillas_result.get("porcentaje", None),
    "muletillas_detectadas": muletillas_result["muletillas_usadas"]
}])

print("\n### 1. Muletillas \n")

pd.set_option("display.max_colwidth", None)

display(df_muletillas)

df_claridad = pd.DataFrame([{
    "puntuacion": claridad_result["puntuacion"],
    "penalizacion": claridad_result["penalizacion"],
    "penalizacion_negativas": claridad_result["penalizacion_negativas"], 
    "penalizacion_frases_largas": claridad_result["penalizacion_frases_largas"], 
    "penalizacion_anglicismos": claridad_result["penalizacion_anglicismos"], 
    "bonificacion": claridad_result["bonificacion"],
    "num_frases_largas": claridad_result["num_frases_largas"], 
    "positivas": claridad_result["positivas"], 
    "negativas": claridad_result["negativas"], 
    "anglicismos": claridad_result["anglicismos"],
    "frases_largas": claridad_result["frases_largas"]
}])

print("\n### 2. Claridad \n")
display(df_claridad)

# 2. Participación DataFrame
df_participacion = pd.DataFrame([{
    "puntuacion": participacion_result["puntuacion"],
    "bonificacion": participacion_result.get("bonificacion", None),
    "penalizacion": participacion_result.get("penalizacion", None),
    "palabras_cliente": participacion_result["palabras_cliente"],
    "palabras_vendedor": participacion_result["palabras_vendedor"],
    "ratio": participacion_result["ratio"],
    "escucha_activa": participacion_result["escucha_activa"],
    "n_escuchas": participacion_result["n_escuchas"]
}])
print("\n### 3. Participacion \n")
display(df_participacion)

# 3. Cobertura de Temas DataFrame
df_cobertura_temas = pd.DataFrame([{
    "puntuacion": cobertura_temas_result["puntuacion"],
    "temas_olvidados": cobertura_temas_result.get("temas_olvidados", None),
    "proximos_pasos": cobertura_temas_result.get("proximos_pasos", None),
    "penalizacion": cobertura_temas_result.get("penalizacion", None),
    "bonificacion": cobertura_temas_result.get("bonificacion", None),
    "señales_temas": cobertura_temas_result.get("señales_temas", None),
    "señales_proximos_pasos": cobertura_temas_result.get("señales_proximos_pasos", None)
}])
print("\n### 4. Temas \n")
display(df_cobertura_temas)

# 4. Índice de Preguntas DataFrame
df_preguntas = pd.DataFrame([{
    "puntuacion": preguntas_result["puntuacion"],
    "total_preguntas": preguntas_result["total_preguntas"],
    "preguntas": preguntas_result["preguntas"],
    "cerradas": preguntas_result["cerradas"],
    "sondeo": preguntas_result["sondeo"],
    "irrelevantes": preguntas_result["irrelevantes"],
    "penalizacion": preguntas_result["penalizacion"],
    "bonificacion": preguntas_result["bonificacion"]
}])
print("\n### 5. Preguntas \n")
display(df_preguntas)

# 5. PPM DataFrame
df_ppm = pd.DataFrame([{
    "puntuacion": ppm_result["puntuacion"],
    "ppms": ppm_result["ppms"],
    "media_ppm": ppm_result["media_ppm"],
    "variabilidad": ppm_result["variabilidad"],
    "penalizacion": ppm_result["penalizacion"],
    "bonificacion": ppm_result["bonificacion"]
}])
print("\n### 6. PPM\n")
display(df_ppm)

# 6. Objetivo
df_obj = pd.DataFrame([{
    "puntuacion": obj_result["puntuacion"],
    "señales": obj_result["señales"]
}])
print("\n### 7. Objetivo\n")
display(df_obj)


### 1. Muletillas 



,puntuacion,penalizacion,total_muletillas,repeticion_constante,porcentaje_repeticion,muletillas_detectadas
0,15,85,17,False,0.176471,"[este, he, ehh, pues, si, va, se, bueno, si, ehh, pues, ehh, este, he, venga, se, si]"



### 2. Claridad 



,puntuacion,penalizacion,penalizacion_negativas,penalizacion_frases_largas,penalizacion_anglicismos,bonificacion,num_frases_largas,positivas,negativas,anglicismos,frases_largas
0,100,15,0,15,0,30,1,"[garantía, capacidad, calidad]",[],[],"[es como se pronuncia en español, tenemos un gustavo, en la boca rindo recibe una bagus vez, rayitos de una pequeña red, pow, muy fuera de la tabla me después al hacer un]"



### 3. Participacion 



...

,puntuacion,total_preguntas,preguntas,cerradas,sondeo,irrelevantes,penalizacion,bonificacion
0,100,2,"[¿qué te parece?, ¿algo más que quieras saber?]",0,0,0,0,0



### 6. PPM



,puntuacion,ppms,media_ppm,variabilidad,penalizacion,bonificacion
0,0,"[18.0, 48.0, 186.0, 96.0, 120.0, 30.0, 60.0, 54.0, 30.0, 48.0, 42.0, 30.0, 78.0, 96.0, 90.0, 168.0, 30.0, 198.0, 24.0, 30.0]",73.8,54.0,115,0



### 7. Objetivo



,puntuacion,señales
0,0,"El vendedor no logra cerrar la venta del coche Mike XL, ya que la conversación termina sin un acuerdo definitivo, y la clienta solo expresa interés en obtener una propuesta y repasar la información."


In [6]:

# Get scores for each KPI using the 'puntuacion' column from each DataFrame
kpi_scores = {
    'muletillas_pausas': df_muletillas['puntuacion'].iloc[0],
    'claridad': df_claridad['puntuacion'].iloc[0],
    'participacion': df_participacion['puntuacion'].iloc[0],
    'cobertura': df_cobertura_temas['puntuacion'].iloc[0],
    'preguntas': df_preguntas['puntuacion'].iloc[0],
    'ppm': df_ppm['puntuacion'].iloc[0]
}

# Define weights as in get_conver_scores.py
pesos = {
    "muletillas_pausas": 0.15,
    "claridad": 0.15,
    "participacion": 0.25,
    "cobertura": 0.15,
    "preguntas": 0.20,
    "ppm": 0.10
}

# Compute the weighted final score using the local DataFrame KPI scores
# Ensure the metric order matches the weights' keys
puntuacion_final = (
    kpi_scores['muletillas_pausas'] * pesos['muletillas_pausas'] +
    kpi_scores['claridad'] * pesos['claridad'] +
    kpi_scores['participacion'] * pesos['participacion'] +
    kpi_scores['cobertura'] * pesos['cobertura'] +
    kpi_scores['preguntas'] * pesos['preguntas'] +
    kpi_scores['ppm'] * pesos['ppm']
)

# Show rounded final score
print(f'Puntuacion final: {puntuacion_final:.1f}')


Puntuacion final: 60.8


## Summary

All metrics have been calculated and displayed in their respective DataFrames above.
